In [0]:
jdbcHostname = "server4traffic.database.windows.net"
jdbcDatabase = "datawarehouse04"
jdbcPort = 1433
jdbcUsername = "admin2traffic"
jdbcPassword = "abcd1234@"
# Dans un vrai projet, il ne faut jamais exposer ce genre d'information dans le code.

In [0]:
# URL JDBC
jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

# Paramètres de connexion
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
dbutils.widgets.text("date_param", "")
date_a_traiter = dbutils.widgets.get("date_param")

In [0]:
# construire la requête SQL avec filtre
query = f"(SELECT * FROM raw.visiteurs WHERE CAST(timestamp_insertion AS DATE) = '{date_a_traiter}') AS visiteurs_filtrés"

# lire uniquement les données de la veille
df_visiteurs = spark.read.jdbc(
    url=jdbcUrl,
    table=query,
    properties=connectionProperties
)

In [0]:
import pyspark.sql.functions as F

In [0]:
# Lire Toute la table SQL dans un DataFrame Spark
# df_visiteurs = spark.read.jdbc(url=jdbcUrl, table="raw.visiteurs", properties=connectionProperties)

In [0]:
display(df_visiteurs.limit(10))

In [0]:
# Compter les lignes où AU MOINS UNE colonne est null
nb_lignes_null = df_visiteurs.filter(
    F.col("store_id").isNull() |
    F.col("sensor_id").isNull() |
    F.col("nb_visiteurs").isNull() |
    F.col("date").isNull() |
    F.col("heure").isNull()
).count()

In [0]:
print(f"Nombre de lignes avec au moins un null : {nb_lignes_null}")

In [0]:
df_visiteurs = df_visiteurs.dropna()


In [0]:
# Compter les lignes où AU MOINS UNE colonne est null
nb_lignes_null = df_visiteurs.filter(
    F.col("store_id").isNull() |
    F.col("sensor_id").isNull() |
    F.col("nb_visiteurs").isNull() |
    F.col("date").isNull() |
    F.col("heure").isNull()
).count()
print(f"Nombre de lignes avec au moins un null : {nb_lignes_null}")

In [0]:
df_visiteurs = df_visiteurs.withColumn("nb_visiteurs", F.col("nb_visiteurs").cast("int"))

In [0]:
df_visiteurs.select("nb_visiteurs").describe().show()

# **SQL**

In [0]:
# Enregistrer le DataFrame comme une table temporaire
df_visiteurs.createOrReplaceTempView("visiteurs")

**Nettoyer la colonne Date**


In [0]:
spark.sql("""
    select distinct date
    from visiteurs
    where date like "%/%/%"
""").show()

In [0]:
spark.sql("""
    select distinct date
    from visiteurs
    where date like "%-%/%"
""").show()

In [0]:
spark.sql("""
    select distinct date
    from visiteurs
    where date like "%-%-2024"
""").show()

In [0]:
clean_visiteurs = spark.sql("""
    SELECT 
        store_id, 
        sensor_id, 
        COALESCE(
            TO_DATE(date, 'yyyy-MM-dd'), 
            TO_DATE(date, 'yyyy/MM/dd'),
            TO_DATE(date, 'dd/MM/yyyy')
        ) AS date, 
        hour(CAST(heure AS TIMESTAMP)) AS heure, 
        nb_visiteurs
    FROM visiteurs
""")
clean_visiteurs.createOrReplaceTempView("clean_visiteurs")
display(clean_visiteurs)

In [0]:
clean_visiteurs = spark.sql("""
    SELECT 
        *
    FROM clean_visiteurs v
    WHERE v.store_id <= 10 AND v.sensor_id <= 12
    AND v.heure BETWEEN 9 AND 19
    AND v.date is not NULL
    AND v.nb_visiteurs BETWEEN 1 and 120 -- eleminer valeurs null et abiraintes
""")
clean_visiteurs.createOrReplaceTempView("clean_visiteurs")
display(clean_visiteurs)

In [0]:
spark.sql("""
    -- table des dates attendues
    WITH dates_attendues AS (
        select d.date from
        (SELECT explode(sequence(DATE '2023-04-27', DATE '2025-04-27', 
        INTERVAL 1 DAY)
        ) AS date) as d
        where dayofweek(d.date) BETWEEN 2 AND 6
    ),
    -- ids des magasins et des capteurs
    identifiants_uniques AS (
        SELECT DISTINCT store_id, sensor_id
        FROM clean_visiteurs
    ),
    toutes_combinaisons AS (
        SELECT i.store_id, i.sensor_id, d.date
        FROM identifiants_uniques i
        CROSS JOIN dates_attendues d
    )

    select t.store_id, t.sensor_id, t.date
    from toutes_combinaisons t
    left join clean_visiteurs v
    on v.store_id = t.store_id
    and v.sensor_id = t.sensor_id
    and v.date = t.date
    where v.date is null
""").show()


On a tout les dates maintenat on va chercher les heures

Trouver les données manquantes

In [0]:
visiteurs_manquants = spark.sql("""
    -- table des dates attendues
    WITH heures_attendues AS (
        SELECT explode(sequence(9, 19)) AS heure
    ), 
    dates_attendues AS (
        select d.date from
        (SELECT explode(sequence(DATE '2023-04-27', DATE '2025-04-27', 
        INTERVAL 1 DAY)
        ) AS date) as d
        where dayofweek(d.date) BETWEEN 2 AND 6
    ),
    -- ids des magasins et des capteurs
    identifiants_uniques AS (
        SELECT DISTINCT store_id, sensor_id
        FROM clean_visiteurs
    ),
    toutes_combinaisons AS (
        SELECT i.store_id, i.sensor_id, d.date,h.heure
        FROM identifiants_uniques i
        CROSS JOIN dates_attendues d
        CROSS JOIN heures_attendues h
    )

    select t.store_id, t.sensor_id, t.date,t.heure
    from toutes_combinaisons t
    left join clean_visiteurs v
    on v.store_id = t.store_id
    and v.sensor_id = t.sensor_id
    and v.date = t.date
    and v.heure = t.heure
    where v.heure is null

""")
visiteurs_manquants.createOrReplaceTempView("visiteurs_manquants")
display(visiteurs_manquants)


In [0]:
visiteurs_manquants = spark.sql("""
    with moyenne_visiteurs as (
    select v.store_id, v.sensor_id, v.date, cast(avg(v.nb_visiteurs) AS int) as nb_visiteurs
    from clean_visiteurs v
    group by v.store_id, v.sensor_id, v.date
    )
    select v.*, mv.nb_visiteurs
    from visiteurs_manquants v
    inner join moyenne_visiteurs mv
    on v.store_id = mv.store_id
    and v.sensor_id = mv.sensor_id
    and v.date = mv.date

""")
visiteurs_manquants.createOrReplaceTempView("visiteurs_manquants")
display(visiteurs_manquants)

In [0]:
clean_visiteurs = spark.sql("""
    SELECT store_id, sensor_id, date, heure, nb_visiteurs
    FROM clean_visiteurs

    UNION ALL

    SELECT store_id, sensor_id, date, heure, nb_visiteurs
    FROM visiteurs_manquants
""")
display(clean_visiteurs)


PySpark

In [0]:
clean_visiteurs = clean_visiteurs \
    .withColumn("store_id", F.col("store_id").cast("int")) \
    .withColumn("sensor_id", F.col("sensor_id").cast("int"))


Check des doublons

In [0]:
clean_visiteurs.groupBy("date", "heure", "store_id", "sensor_id") \
    .agg(F.count("*").alias("nb")) \
    .filter(F.col("nb") > 1) \
    .show()

In [0]:
clean_visiteurs.count()

In [0]:
clean_visiteurs = clean_visiteurs.dropDuplicates(
    ["date", "heure", "store_id", "sensor_id"]
)

In [0]:
clean_visiteurs.count()

In [0]:
clean_visiteurs.select(F.min(F.col("date")), F.max(F.col("date"))).show()

In [0]:
clean_visiteurs.select("nb_visiteurs").describe().show()

In [0]:
display(clean_visiteurs)

In [0]:
try:
    clean_visiteurs.write.jdbc(
        url=jdbcUrl,
        table="analytics.visiteurs",
        mode="append",  # ou "overwrite" selon usage
        properties=connectionProperties
    )
    print("Données insérées avec succès dans analytics.visiteurs.")
except Exception as e:
    print("Erreur lors de l'insertion dans la base SQL :")
    print(str(e))